### ANIL FERDI KAYA
### 090180128
### BVA529E ASSESMENT3

In [2]:
!pip install gym==0.21.0
!pip install pyglet

  Using cached gym-0.21.0.tar.gz (1.5 MB)
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616820 sha256=f56cc5d5f006553cc65fd64e9e98ab3f585d15cf8b720f209cc5ee08336f206e
  Stored in directory: /home/anilk/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Using cached pyglet-1.5.26-py3-none-any.whl (1.1 MB)


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
import time
import gym

# SPECIFIATIONS

In [4]:
env = gym.make("CartPole-v0")
env.reset()

array([-0.02125209,  0.04118586,  0.03255917, -0.0498687 ], dtype=float32)

In [7]:
def merge_state_action(s, a):
    return np.concatenate((s, [a]))

In [8]:
def generate_samples(n_episodes=10000):
    samples = []
    for i in range(n_episodes):
        s = env.reset()
        done=False
        while not done:
            a=env.action_space.sample() #random action
            sa = merge_state_action(s, a)
            samples.append(sa)
            s, r, done, info = env.step(a)
    return samples

In [9]:
samples = generate_samples()

In [10]:
rbf_sampler = RBFSampler(n_components=100)
rbf_sampler.fit(samples)

RBFSampler()

In [13]:
def predictQ(s,a,w):
    sa = merge_state_action(s, a)
    x = rbf_sampler.transform([sa])[0]
    return np.dot(x,w)

In [14]:
def gradientQ(s,a,w):
    sa = merge_state_action(s, a)
    x = rbf_sampler.transform([sa])[0]
    return x

In [15]:
def predictQ_all_actions(s,w):
    values=[]
    for a in range(env.action_space.n):
        values.append(predictQ(s,a,w))
    return values

In [20]:
def epsilon_greedy(s, w, eps=0.1):
    values=[]
    p = np.random.random()
    if p < (1 - eps):
        values = predictQ_all_actions(s,w)
        return np.argmax(values)
    else:
        return env.action_space.sample()

In [47]:
def watch_agent(w,LAMBDA):
    done = False
    episode_reward = 0
    s = env.reset()
    while not done:
        a = epsilon_greedy(s, w,eps=0)
        s, r, done, info = env.step(a)
        time.sleep(0.02)
        env.render()
        episode_reward += r
    print('Watkins Q with lambda '+str(LAMBDA)+' reward: ',episode_reward)

# Watkins Q Implemantation

<img src="https://i.stack.imgur.com/lhI2Y.png"/>


### When i create a algorithm i upper picture  and lecture 8 num 14 slide

In [50]:
def watkinsQ(GAMMA = 0.9,ALPHA=0.1,LAMBDA=0,NUM_EPISODES = 1000):
    w = np.zeros(100) 

    for it in range(NUM_EPISODES):
        s=env.reset()
        e=np.zeros(100)
        
        done=False
        
        while not done:
            a = epsilon_greedy(s ,w,eps=0.1)
            s2, r, done, info = env.step(a)
            if done:
                target = r
            else:
                values = predictQ_all_actions(s2,w)
                sa = merge_state_action(s, a)
                target = r + GAMMA * np.max(values)               
            # update the weights
            g = gradientQ(s,a,w)
            err=target - predictQ(s,a,w)
            e = GAMMA*LAMBDA*e + g
            w += ALPHA * err * e
            # update state
            s = s2  
    watch_agent(w,LAMBDA)
    env.close()

# RUN

In [54]:
np.random.seed(14)
watkinsQ(LAMBDA=0)
watkinsQ(LAMBDA=0.95)
watkinsQ(LAMBDA=1)

Watkins Q with lambda 0 reward:  198.0
Watkins Q with lambda 0.95 reward:  172.0
Watkins Q with lambda 1 reward:  110.0
